In [4]:
import csv
from scraper import WebsiteScraper

def load_urls_from_csv(file_path, column_name):
  urls = []
  with open(file_path, mode='r') as file:
      csv_reader = csv.DictReader(file)
      for row in csv_reader:
          if column_name in row:
              urls.append(row[column_name])
  return urls

def print_summaries_from_websites(urls):
  for url in urls:
    if url: 
      print (f"\n{url}")
      scraper = WebsiteScraper(url)
      connectivity = scraper.test_connectivity()
      print(connectivity)
        
csv_path = 'data/organizations.csv'

urls = [
  # head, get, reason, expected
  'https://fullyfunctional.com.au/', # 405, 200, parked page from registrar, display 405 [x]
  'http://qubecell.com', # 405, 200, redirect to subdomain parked, display 405 [x]
  'http://www.albert2005.co.jp/', # max connection error, max connection error, redirect to external path, catch redirect [ ]
  'https://criticalforce.fi/presskit/', # 404, 404, page unexistent but domain active, display 404 [x]
  'http://www.indovance.com', # head 404, get 200, seems ok, retry on 404 [x]
  'http://specdox.com', # 200, n/a, redirects, catch redirect [x]
  'http://foxonline.com', # 403, 403, redirect to external path, catch redirect & display 403 [x]
]

# urls = load_urls_from_csv(csv_path, 'homepage_url')
print_summaries_from_websites(urls)


https://fullyfunctional.com.au/
[Site inaccessible: 405 Client Error: Method Not Allowed for url: https://fullyfunctional.com.au/]

http://qubecell.com
[Site inaccessible: 405 Client Error: Not Allowed for url: http://ww16.qubecell.com/?sub1=20240714-0904-3649-94ef-906323ceb19e]

http://www.albert2005.co.jp/
[Server unreachable]

https://criticalforce.fi/presskit/


HTTPConnectionPool(host='www.albert2005.co.jp', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11125be90>: Failed to establish a new connection: [Errno 61] Connection refused'))


[Site inaccessible: 404 Client Error: Not Found for url: https://criticalforce.fi/presskit/]

http://www.indovance.com
[Site accessible: OK]

http://specdox.com
[specdox.com -> scriptpreneur.com]: [Site accessible: OK]

http://foxonline.com
[foxonline.com -> abracon.com]: [Site inaccessible: 403 Client Error: Forbidden for url: https://abracon.com/brands/fox?utm_source=foxonline.com&utm_medium=301]
